In [42]:
import numpy as np
import mpmath 
import sympy
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from scipy import optimize

ImportError: No module named 'mpmath'

In [12]:
theta = 2.0
beta = 0.99
rho = - np.log(beta)
A0 = 1.0
g = 0.005 
L0 = 100
n = 0.0025 
H = 1.0
delta = 0.025 
alpha = 0.33

In [13]:
rho , n

(0.010050335853501451, 0.0025)

In [14]:
rho - n - (1 - theta) * g > 0

True


###### In the cell below I have provided code for the consumption Euler equation as well as code for two implicit functions defining the behavior of $k$ and $c$ in the neigborhood of the steady state of the Ramsey economy. The rest of the functions are as follows:

###### * the intensive form of the Cobb-Douglas production function, $f(k_{t})$
###### * the evolution of capital per effective worker, $k_{t+1}$, in terms of $k_{t}$ and $c_{t}$.
###### * the implicit function describing the evolution of capital per effective worker in the neighborhood of the steady state.
###### * the implicit function describing the consumption euler equation in the neighborhood of the steady state.
# 
###### You all should be familiar enough with the intensive form of the production function. Here is the equation of motion for capital:
 
### $$k_{t+1} = e^{-(n + g)}(f(k) + e^{-\delta}k_{t} - c_{t}) $$


In [15]:
def f(k):
    """
    
    In your lectures you will have been introduced to the intensive form of 
    the production function.  The intensive form of the production function 
    expresses output per effective worker, y, as a function of the capital 
    stock per effective worker, k.
    
    Inputs:
        1) k_t: current period's level of capital per effective worker
        
    Returns:
        1) y_t: current period's output per effective worker
        
    """
    return k**alpha

def euler(k, c):
    """
    
    Via the consumption Euler equation, next period's consumption per
    effective worker can be written as a function of current period 
    consumption and capital stock (both per effective worker).
    
    Inputs:
        1) k_t: next period's level of capital per effective worker
        2) c_t: current period's level of consumption per effective worker
        
    Returns:
        1) c_t+1: next period's consumption per effective worker
        
    """
    return np.exp(-g) * (np.exp(-rho) * (np.exp(-delta) + alpha * capital(k, c)**(alpha - 1)))**(1 / theta) * c
    
def capital(k, c):
    """
    
    Next period's capital stock per effective worker can be written as a 
    function of current period consumption and capital stock (both per 
    effective worker).
    
    Inputs:
        1) k_t: current period's level of capital per effective worker
        2) c_t: current period's level of consumption per effective worker
        
    Returns:
        1) k_t+1: next period's capital stock per effective worker
        
    """
    return np.exp(-(n + g)) * (f(k) + np.exp(-delta) * k - c)

def F1(k, c):
    """
    
    Can express the change in capital stock per effective worker between 
    any two periods as follows:
        
        k_t+1 - k_t = capital(k, c) - k_t
    
    In steady-state, capital stock per effective worker must be constant, 
    and thus the LHS of the above must be zero. This implies that, the 
    steady state values of k and c will solve the following equation:
    
        capital(k, c) - k = 0
    
    """
    return capital(k, c) - k
    
def F2(k, c):
    """
    
    Can express the change in consumption per effective worker between 
    any two periods as follows:
        
        c_t+1 - c_t = euler(k, c) - c
    
    In steady-state, consumption per effective worker must be constant, 
    and thus the LHS of the above must be zero. This implies that, the 
    steady state values of k and c will solve the following equation:
    
        euler(k, c) - c = 0
    
    """
    return euler(k, c) - c


In [16]:
def k_star(): 
    """
    
    The steady-state level of capital stock per effective worker, k_star, 
    in the Ramsey model is a function of the exogenous parameters!
    
    N.B.: The function takes no arguments because parameters are defined above!
    """
    return ((alpha * np.exp(-rho)) / (np.exp(theta * g) - np.exp(-(rho + delta))))**(1 / (1 - alpha))
    
def c_star(): 
    """
    
    The steady-state level of consumption per effective worker, c_star, 
    in the Ramsey model is a direct function of the exogenous parameters 
    and the steady-state level of capital stock per effective worker 
    (which is itself a function of only exogenous parameters).
    
    N.B.: The function takes no arguments because parameters are defined above!
    """
    return f(k_star()) + (np.exp(-delta) - np.exp(n + g)) * k_star()


In [17]:
k_star(), c_star()

(19.60285758306458, 2.0381476236391665)

##### Here I use the 'fsolve' function to find find the steady-state values of k and c numerically.  
##### Equations **capital** and **euler** form a coupled two equation dynamical system in two unknowns.  
##### Above we have defined two implicit functions F1 and F2 such that:
#     
######     F1(k_Star, c_Star) = 0
######     F2(k_Star, c_Star) = 0
# 
##### Thus, mathematically, the problem of finding the steady-state values of k and c, is "simply" a root-finding problem! 
##### To find the root of this equation we will again use the function 'fsolve()' from the scipy.optimize package.

In [18]:
def ramseySteadyState(X):
    out = [F1(X[0], X[1])]
    out.append(F2(X[0], X[1]))
    return out

# Apply fsolve()...note that we need to give the algorithm a decent guess!
k0, c0 = 3, 1
k_Star, c_Star = optimize.fsolve(func=ramseySteadyState, x0=(k0, c0))

In [19]:
k_Star, c_Star

(19.602857582146562, 2.0381476236311964)

In [23]:
# Display numerical steady-state values and compare to analytic solutions
print("Numeric  k*:  ", k_Star)
print("Analytic k*:  ", k_star())
print("Numeric  c*:  ", c_Star)
print("Analytic c*:  ", c_star())

Numeric  k*:   19.6028575821
Analytic k*:   19.6028575831
Numeric  c*:   2.03814762363
Analytic c*:   2.03814762364


In [25]:
""""
Simulating the deterministic Ramsey model
"""

class ramseyModel(object):
    
    def __init__(self, params, k=None, c=None):
        """
        
        Represents the Ramsey model 
        
        Definition of exogenous parameters of the Ramsey Model
            1) theta: coefficient of relative risk aversion 
            2) beta: discount factor
            3) rho: discount rate
            4) A0: initial level of technology
            5) g: growth rate of technology
            6) L0: initial level of population
            7) n: population growth rate
            8) H: household size
            9) delta: rate of capital depreciation
            10) alpha: capital share of output
        Attributes: 
            1) params: a dictionary of parameters and their values (i.e., {'alpha':0.33, ...}.
            2) k: an initial condition for the state variable k (i.e., capital per
                  effective worker!)
            3) c: an initial condition for the control variable c (i.e., consumption per
                  effective worker!)
        
        """
        # current value of state variable, k
        self.k            = k
        # current value of the control variable, c
        self.c            = c
        # dictionary of parameter values
        self.param_dict   = params
        # dictionary of steady state values        
        self.SS_dict      = {'k_star':self.set_k_star(self.param_dict), 
                             'c_star':self.set_c_star(self.param_dict),
                             's_star':self.set_s_star(self.param_dict)}
    def u(self, C):
        """
        Representative agent has constant relative risk aversion (CRRA)
        preferences over consumption per worker, C.
        For \theta \neq 1, CRRA preferences are:
        u(C_{t}) = \frac{C_{t}^{1 - \theta} - 1}{1 - \theta}
        For \theta = 1, CRRA preferences are equivalent to:
        u(C_{t} = ln C_{t}
        Inputs: consumption per worker, C.
        Returns: utility from consuming C, u(C).
        """
        # extract the params
        theta = self.param_dict['theta']
        
        if theta != 1:
            return (C**(1 - theta) - 1) / (1 - theta)
        else:
            return np.log(C)
    
    def lifetimeUtility(self, c):
        """
        Computes lifetime utility associated with a consumption stream. Note
        that the function automatically accounts for growth in technology and
        population.
        Inputs: an array, c representing a consumption stream in per effective
        worker units.
        Returns: A list containing...
            1. Present discount value, in terms of utility, of the consumption
            stream.
            2. The path of discounted flow utility (for plotting)
        """
        # extract the params
        A0   = self.param_dict['A0']
        g    = self.param_dict['g']
        L0   = self.param_dict['L0']
        H    = self.param_dict['H']
        n    = self.param_dict['n']
        rho  = self.param_dict['rho']
        
        # need to inflate consumption per effective worker by technology
        tech_path = A0 * np.exp(g * np.arange(0, np.size(c), 1))
        
        # need to discount future utility from consumption appropriately
        discounted_u = (L0 / H) * np.exp((n - rho) * np.arange(0, np.size(c), 1))
        
        # compute the path of utility
        utility_path = discounted_u * self.u(c * tech_path)
        
        return [np.sum(utility_path), utility_path]

    def set_k_star(self, params): 
        """
    
        The steady-state level of capital stock per effective worker, k_star, 
        in the Ramsey model is a function of the exogenous parameters!
    
        """
        # extract params
        n     = params['n']
        g     = params['g']
        alpha = params['alpha']
        delta = params['delta']
        rho   = params['rho']
        theta = params['theta']
    
        return ((alpha * np.exp(-rho)) / (np.exp(theta * g) - np.exp(-(rho + delta))))**(1 / (1 - alpha))
    
    def set_c_star(self, params): 
        """
    
        The steady-state level of consumption per effective worker, c_star, 
        in the Ramsey model is a direct function of the exogenous parameters 
        and the steady-state level of capital stock per effective worker 
        (which is itself a function of only exogenous parameters).
    
        """
        # extract params
        n      = params['n']
        g      = params['g']
        alpha  = params['alpha']
        delta  = params['delta']
        k_star = self.set_k_star(params)
        
        return k_star**alpha + (np.exp(-delta) - np.exp(n + g)) * k_star
    
    def set_s_star(self, params):
        """ 
        Steady state savings rate of the Ramsey economy
    
        """
        # extract params
        n     = params['n']
        g     = params['g']
        alpha = params['alpha']
        delta = params['delta']
        rho   = params['rho']
        theta = params['theta']
        
        s     = alpha * np.exp(-rho) * (np.exp(n + g) - np.exp(-delta)) / \
                (np.exp(theta * g) - np.exp(-(rho + delta)))
        return s
    
    def capital(self, k, c):
        """
    
        Next period's capital stock per effective worker can be written as a 
        function of current period consumption and capital stock (both per 
        effective worker).
    
        Inputs:
            1) k_t: current period's level of capital per effective worker
            2) c_t: current period's level of consumption per effective worker
        
        Returns:
            1) k_t+1: next period's capital stock per effective worker
        
        """
        # extract params
        n     = self.param_dict['n']
        g     = self.param_dict['g']
        alpha = self.param_dict['alpha']
        delta = self.param_dict['delta']
    
        return np.exp(-(n + g)) * (k**alpha + np.exp(-delta) * k - c)
    
    def euler(self, k, c):
        """
    
        Via the consumption Euler equation, next period's consumption per
        effective worker can be written as a function of current period 
        consumption and capital stock (both per effective worker).
    
        Inputs:
            1) k_t: next period's level of capital per effective worker
            2) c_t: current period's level of consumption per effective worker
        
        Returns:
            1) c_t+1: next period's consumption per effective worker
        
        """
        # extract params
        n     = self.param_dict['n']
        g     = self.param_dict['g']
        alpha = self.param_dict['alpha']
        delta = self.param_dict['delta']
        rho   = self.param_dict['rho']
        theta = self.param_dict['theta']
    
        return np.exp(-g) * (np.exp(-rho) * (np.exp(-delta) + alpha * capital(k, c)**(alpha - 1)))**(1 / theta) * c

    def update(self):
        """
        
        Update the state variables according to k_{t+1} = capital(k_t, c_t, params) and 
        c_{t+1} = euler(k_t, c_t, params).
        
        CRUCIAL that k is updated PRIOR to updating c!
        """
        # remember to always update k first!
        self.k = self.capital(self.k, self.c) 
        self.c = self.euler(self.k, self.c)

    def sample_path(self, n=None):
        """
        
        Generate sample path of length n starting from the current state
        
        """
        path = np.zeros(shape=(n, 2))
        
        for t in range(n):
            path[t, 0] = self.k
            path[t, 1] = self.c
            self.update()
        
        return path

    def steady(self, k0=None, c0=None):
        """
        
        Finds the steady state for the Ramsey economy either using the 
        analytic solution, or using a root solver and an initial guess.
        
        """
        
        def ramseySS(X):
            out = [self.capital(X[0], X[1]) - X[0]]
            out.append(self.euler(X[0], X[1]) - X[1])
            return out

        return optimize.fsolve(func=ramseySS, x0=(k0, c0))
        
    def stability(self):
        """
        
        Computes the Jacobian matrix of partial derivatives (evaluated at steady-state), 
        and then calculates the Jacobian's eigenvalues and eigenvectors.
        
        """
        
        # calculate partial derivatives
        capital_c = mp.diff(f=self.capital, x=(self.SS_dict['k_star'], self.SS_dict['c_star']), n=(0, 1))
        capital_k = mp.diff(f=self.capital, x=(self.SS_dict['k_star'], self.SS_dict['c_star']), n=(1, 0))
        euler_c   = mp.diff(f=self.euler, x=(self.SS_dict['k_star'], self.SS_dict['c_star']), n=(0, 1))
        euler_k   = mp.diff(f=self.euler, x=(self.SS_dict['k_star'], self.SS_dict['c_star']), n=(1, 0))
        
        # define the Jacobian
        jacobian = np.array([[capital_k, capital_c], [euler_k, euler_c]], dtype='float')
        
        # calculate eigenvalues/vectors
        eigenvalues, eigenvectors = np.linalg.eig(jacobian)
        
        return [jacobian, eigenvalues, eigenvectors]
    
    def get_lifetimeUtility(self, c, A0=1, L0=1, H=1):
        """
        Computes lifetime utility associated with a consumption
        stream. Note that the function automatically accounts for
        growth in technology and population.
        Inputs:
            1. an array, c representing a consumption stream in per
                effective worker units.
            2. A0: (default = 1) some value for the initial level of
                technology.
            3. L0: (default = 1) some value for the initial size of
                labor force.
            4. H: (default = 1) size of the representative household.
        Returns: A list containing...
            1. Present discount value, in terms of utility, of the
                consumption stream.
            2. The path of discounted flow utility (for plotting)
        """
        # extract the params
        g = self.param_dict['g']
        n = self.param_dict['n']
        beta = self.param_dict['beta']
        
        # time path
        time = np.arange(0, np.size(c), 1)
        
        # need to inflate consumption per effective worker by technology
        tech_path = A0 * np.exp(g * time)
        
        # need to discount future utility from consumption
        discount_factor = (L0 / float(H)) * (beta * n)**time
        
        # compute the path of utility
        utility_path = discount_factor * self.get_utility(c * tech_path)
        
        return [np.sum(utility_path), utility_path]
    
    def forward_shoot(self, k0=None, c0=None, tol=1.5e-08):
        """
        
        Computes the full, non-linear saddle path for the Ramsey model using the 
        'forward shooting' algorithm (see Judd (1992) p. 357 for details). 
        
        """
        # compute steady state values
        k_star, c_star = self.SS_dict['k_star'], self.SS_dict['c_star']
        
        if k0 <= k_star:
            c_l = 0
            c_h = c_star
        else:
            c_l = c_star
            c_h = k0**alpha
        c0 = (c_h + c_l) / 2
        self.k, self.c = k0, c0
    
        # Initialize a counter
        count  = 0
        n_iter = 0
        
        # Forward Shooting Algorithm
        while 1:
            self.update()
            dist = np.abs(((self.k - k_star)**2 + (self.c - c_star)**2)**0.5)
            count = count + 1
            if k0 <= k_star:
                if self.k > k_star:
                    if dist < tol:
                        break
                    else: # initial c_l too low!
                        c_l = c0
                        c0 = (c_h + c_l) / 2
                        self.k, self.c = k0, c0
                        count = 0
                if self.c > c_star:
                    if dist < tol:
                        break
                    else: # initial c_h too high!
                        c_h = c0 
                        c0 = (c_h + c_l) / 2
                        self.k, self.c = k0, c0
                        count = 0
            else:
                if self.k < k_star:
                    if dist < tol:
                        break
                    else: # initial c_l too high!
                        c_h = c0 
                        c0 = (c_h + c_l) / 2
                        self.k, self.c = k0, c0
                        count = 0
                if self.c < c_star:
                    if dist < tol:
                        break
                    else: # initial c_l too low!
                        c_l = c0
                        c0 = (c_h + c_l) / 2
                        self.k, self.c = k0, c0
                        count = 0
                
        self.k, self.c = k0, c0
        solutionPath = self.sample_path(count)

        return [self.c, solutionPath, count, dist]


In [26]:
params = {'theta':theta, 'rho':rho, 'A0':A0, 'g':g, 'L0':L0, 'n':n, 'H':H, 'delta':delta, 'alpha':alpha}

In [27]:
ramsey = ramseyModel(params)

In [28]:
ramsey.k, ramsey.c = 12, 0.4

In [31]:
k, c = sympy.var('k, c')
k_c_sym = sympy.diff(capital(k, c), c).simplify()
k_k_sym = sympy.diff(capital(k, c), k).simplify()
c_c_sym = sympy.diff(euler(k, c), c).simplify()
c_k_sym = sympy.diff(euler(k, c), k).simplify()


# load SymPy printing (needs update)
%load_ext sympyprinting

# Display one of the resulting functions to show the output
print(k_k_sym)

0.327534258090316*k**(-0.67) + 0.968022449831306


C:\Anaconda3\lib\site-packages\IPython\extensions\sympyprinting.py:118: UserWarning: The sympyprinting extension in IPython is deprecated, use 'from sympy import init_printing; init_printing()'
  warnings.warn("The sympyprinting extension in IPython is deprecated, "
C:\Anaconda3\lib\site-packages\sympy\interactive\ipythonprinting.py:49: SymPyDeprecationWarning: 

using %load_ext sympy.interactive.ipythonprinting has been deprecated
since SymPy 0.7.3. Use from sympy import init_printing ;
init_printing() instead. See
https://github.com/sympy/sympy/issues/7013 for more info.

  issue=7013


In [33]:
# Generate a sample path of length 10 for the Ramsey economy using the sample_path() method
ramsey.sample_path(50)

array([[  61.42125699,    0.32582349],
       [  62.99643988,    0.32185225],
       [  64.55760688,    0.31787857],
       [  66.10438873,    0.31390612],
       [  67.63645223,    0.30993826],
       [  69.15349798,    0.30597809],
       [  70.65525828,    0.30202846],
       [  72.1414952 ,    0.29809201],
       [  73.61199869,    0.29417115],
       [  75.0665849 ,    0.29026811],
       [  76.50509453,    0.28638494],
       [  77.9273913 ,    0.28252351],
       [  79.33336056,    0.27868554],
       [  80.72290791,    0.27487263],
       [  82.09595795,    0.27108619],
       [  83.4524531 ,    0.26732757],
       [  84.79235244,    0.26359795],
       [  86.11563072,    0.25989844],
       [  87.42227729,    0.25623002],
       [  88.71229522,    0.2525936 ],
       [  89.9857004 ,    0.24898997],
       [  91.24252068,    0.24541988],
       [  92.48279511,    0.24188396],
       [  93.70657319,    0.2383828 ],
       [  94.91391416,    0.23491692],
       [  96.10488636,   

###### Computing the Jacobian matrix of partial derivatives
###### Method 1: Symbolic differentiation using SymPy. 
###### Method 2: Numerical differentiation using Python. 

In [34]:
# Check that the method steady() returns the same steady-state values obtained above
ramsey.steady(k0=3, c0=1)

array([ 19.60285758,   2.03814762])

##### Evaluate these symbolic derivatives at the steady state values of k_star and c_star and compare the results with those obtained from analytic derivatives

In [36]:
ramseyJacobian = np.array([[k_k_sym.evalf(n=12, subs={k:k_star(), c:c_star()}), k_c_sym.evalf(n=12, subs={k:k_star(), c:c_star()})], 
                             [c_k_sym.evalf(n=12, subs={k:k_star(), c:c_star()}), c_c_sym.evalf(n=12, subs={k:k_star(), c:c_star()})]])
print(ramseyJacobian)

[[1.01262942182 -0.992528054819]
 [-0.00155368796866 1.00152284623]]


In [43]:
import mpmath

capital_c = mp.diff(f=capital, x=(k_Star, c_Star), n=(0, 1))
capital_k = mp.diff(f=capital, x=(k_Star, c_Star), n=(1, 0))
euler_c = mp.diff(f=euler, x=(k_Star, c_Star), n=(0, 1))
euler_k = mp.diff(f=euler, x=(k_Star, c_Star), n=(1, 0))


ramseyJacobian2 = np.array([[capital_k, capital_c], [euler_k, euler_c]], dtype='float')

print(ramseyJacobian2)


ImportError: No module named 'mpmath'

In [39]:
# Compute the eigenvalues and eigenvectors of the Jacobian matrix...
ramseyEigenvalues, ramseyEigenvectors = np.linalg.eig(ramseyJacobian2)

# Display the eigenvalues and eigenvectors...
print ramseyEigenvalues
print ramseyEigenvectors



SyntaxError: Missing parentheses in call to 'print' (<ipython-input-39-e3e3170addb4>, line 5)

In [ ]:
# finally, can use the built in method for computing jacobian and eigenvalues/vectors
ramseyJacobian, ramseyEigenvalues, ramseyEigenvectors = ramsey.stability()


print ramseyJacobian
print ramseyEigenvalues
print ramseyEigenvectors

In [ ]:
# create an instance of the Python object representing the Ramsey model
ramsey = ramseyModel(params)

# pick an initial condition for capital...
k0 = 0.5

# solve the model!
ramsey_solution = ramsey.forward_shoot(k0, tol=1.5e-4)


In [40]:
"""
Plot the full saddle path solution!
"""

def locusK(k):
    """
    
    In order to plot the phase diagram, we need a function that 
    takes k as an input and return the value of c consistent with 
    F1(k, c) = 0! 
    
    Inputs:
        1) k: capital stock per effective worker
        
    Returns:
        1) c: consumption per effective worker 
        
    """
    return f(k) - (np.exp(n + g) - np.exp(-delta)) * k

# Create a grid of points for plotting
gridmax, gridsize = 2000, 100000
grid = np.linspace(0, gridmax, gridsize)

# Create a new plot
plt.figure()

# Add the c and k locii
plt.plot(grid, locusK(grid), '-', color='orange', label=r'$\Delta k=0$')
plt.axvline(k_star(), color='k', label=r'$\Delta c=0$')
plt.plot(k_star(), c_star(), marker='.', markersize=10, color='k')

# Add the initial level of capital per worker
plt.axvline(k0, color='k', ls='--', label=r'$k_{0}$')

# Plot the full saddle-path
plt.plot(ramsey_solution[1][:, 0], ramsey_solution[1][:, 1], color='r')

# Don't forget to label your axes!
plt.xlim(0, 1)
plt.xlabel('k')
plt.ylim(0, 4)
plt.ylabel('c', rotation='horizontal')

# Add a title to the plot
plt.title('Saddlepath of the Ramsey economy')

# Add a legend
plt.legend(frameon=False)
plt.show()

NameError: name 'ramsey_solution' is not defined

###### Welfare Analysis
###### ------------------------
###### Two different ways to analyze welfare in the Ramsey model:
###### 
###### 1. Instantaneous welfare as measured by the flow of utility to the representative individual over time.
###### 2. The present discounted value of expected lifetime consumption.
###### 
###### Here we can calculate measure for the full saddle path.


In [ ]:
plt.plot(ramsey_solution[1][:, 1], color='b', label=r'$c_{Full}$')
plt.axhline(c_star(), ls='--', color='k', label=r'$c^{*}$')
plt.xlim(0, 200)
plt.xlabel('Time')
plt.ylabel('Consumption per effective worker, c')

# Add the legend
plt.legend(loc='best', frameon=False)

# Add a title to the plot
plt.title('Solution paths of consumption per effective worker, c', weight='bold')
plt.show()

 ##### Then we can plot the flow of utility over time

In [ ]:
# Recall that the utility function is defind as follows...
def u(C):
    """
    
    The Ramsey model assumes that the representative individual has CRRA utility.      
    Individual utility is a function of consumption per person, C. The parameter, 
    0 < theta, plays a dual role: 
        
        1) theta is the agent's coefficient of relative risk aversion
        2) 1 / theta is the agent's elasticity of inter-temporal substitution
    
    N.B.: If theta = 1, then CRRA utility is equivalent to log utility! 
    
    Inputs:
        1) C: consumption per worker
        
    Returns:
        1) the value of utility from consumption
        
    """
    if theta != 1:
        return (C**(1 - theta) - 1) / (1 - theta)
    else:
        return np.log(C)

# Create a grid (useful for plotting technology...recall that consumption per worker C = c * A!)
grid = np.arange(0, ramsey_solution[2], 1)

# Create a new plot
plt.figure(figsize=(8,8))
plt.plot(grid, u(ramsey_solution[1][:, 1] * A0 * np.exp(g * grid)), color='b', label=r'$u_{Full}$')

# Add the steady state value for capital per effective worker
plt.plot(grid, u(c_star() * A0 * np.exp(g * grid)), ls='--', color='k', label=r'$u^{*}$')

# Don't forget to label your axes!
plt.xlim(0, ramsey_solution[2])
plt.xlabel('Time')
plt.ylabel('Instantaneous Welfare')

# Add the legend
plt.legend(loc='best', frameon=False)

# Add a title to the plot
plt.title('Flow of welfare (i.e., utility) for representative individual')
plt.show()

In [ ]:
"""
Calculate Hicksian Equivalent Variation
"""

# solve via forward shooting algorith to create a consumption stream
c_forward = ramsey_solution[1][:, 1]        

# create a grid the length of the consumption stream
grid10 = np.arange(0, 11, 1)
grid15 = np.arange(0, 16, 1)
grid20 = np.arange(0, 21, 1)
grid25 = np.arange(0, 26, 1)
grid_full = np.arange(0, ramsey_solution[2], 1)

# U(.) is definied above, if theta=1 then u()= np.log()
X_10 = u(c_forward[:11] * A0 * np.exp(g * grid10))
X_15 = u(c_forward[:16] * A0 * np.exp(g * grid15))
X_20 = u(c_forward[:30] * A0 * np.exp(g * grid20))
X_25 = u(c_forward[:26] * A0 * np.exp(g * grid25))
X_full = u(c_forward * A0 * np.exp(g * grid_full))

Y_10 = u(c_star() * A0 * np.exp(g * grid10))
Y_15 = u(c_star() * A0 * np.exp(g * grid15))
Y_20 = u(c_star() * A0 * np.exp(g * grid20))
Y_25 = u(c_star() * A0 * np.exp(g * grid25))
Y_full = u(c_star() * A0 * np.exp(g * grid_full))

# Initialize the welfare stream
forward_10 = np.zeros(np.size(grid10))
forward_15 = np.zeros(np.size(grid15))
forward_20 = np.zeros(np.size(grid20))
forward_25 = np.zeros(np.size(grid25))
forward_full = np.zeros(np.size(grid_full))

integrated_10 = np.zeros(np.size(grid10))
integrated_15 = np.zeros(np.size(grid15))
integrated_20 = np.zeros(np.size(grid20))
integrated_25 = np.zeros(np.size(grid25))
integrated_full = np.zeros(np.size(grid_full))

# calculate the discounted utility stream
for i in range(np.size(grid10)):
    forward_10[i] = (beta*n)**(i) * X_10[i]
    integrated_10[i] = (beta*n)**(i) * Y_10[i]

for i in range(np.size(grid15)):
    forward_15[i] = (beta*n)**(i) * X_15[i]
    integrated_15[i] = (beta*n)**(i) * Y_15[i]
    
for i in range(np.size(grid20)):
    forward_20[i] = (beta*n)**(i) * X_20[i]
    integrated_20[i] = (beta*n)**(i) * Y_20[i]    
    
for i in range(np.size(grid25)):
    forward_25[i] = (beta*n)**(i) * X_25[i]
    integrated_25[i] = (beta*n)**(i) * Y_25[i]   
    
for i in range(np.size(grid_full)):
    forward_full[i] = (beta*n)**(i) * X_full[i]
    integrated_full[i] = (beta*n)**(i) * Y_full[i]
    
# sum the discounted utility stream to calculate total welfare
Forward_10 = np.sum(forward_10)
Integrated_10 = np.sum(integrated_10)
Forward_15 = np.sum(forward_15)
Integrated_15 = np.sum(integrated_15)
Forward_20 = np.sum(forward_20)
Integrated_20 = np.sum(integrated_20)
Forward_25 = np.sum(forward_25)
Integrated_25 = np.sum(integrated_25)
Forward_welfare = np.sum(forward_full)
Integrated_welfare = np.sum(integrated_full)

# calculate the Hicksian equivalent variation and print
mu_10 = np.e**((1-beta*n)*(1/(1-(beta*n)**11))*(Integrated_10 - Forward_10)) - 1
mu_15 = np.e**((1-beta*n)*(1/(1-(beta*n)**16))*(Integrated_15 - Forward_15)) - 1
mu_20 = np.e**((1-beta*n)*(1/(1-(beta*n)**21))*(Integrated_20 - Forward_20)) - 1
mu_25 = np.e**((1-beta*n)*(1/(1-(beta*n)**26))*(Integrated_25 - Forward_25)) - 1
mu_full = np.e**((1-beta*n)*(Integrated_welfare - Forward_welfare)) - 1


In [ ]:
mu_10, mu_full

In [ ]:
forward_10, integrated_10

In [ ]:
forward_25, integrated_25


In [ ]:
c_forward[:10]


In [ ]:
len(c_forward[:10])

In [ ]:
ramsey_solution
